In [ ]:
import onnxruntime_genai as og

In [ ]:
model_path = '../../Models/Phi-3-vision-128k-instruct-onnx-cpu/cpu-int4-rtn-block-32-acc-level-4'
img_path = './imgs/demo.png'

In [ ]:
model = og.Model(model_path)
processor = model.create_multimodal_processor()
tokenizer_stream = processor.create_stream()

In [ ]:
text = "Please create Python code for image"

In [ ]:
prompt = "<|user|>\n"
prompt += "<|image_1|>\n"
prompt += f"{text}<|end|>\n<|assistant|>\n"

In [ ]:
inputs = processor(prompt, images=image)

In [ ]:
params = og.GeneratorParams(model)
params.set_inputs(inputs)
params.set_search_options(max_length=3072)

In [ ]:
generator = og.Generator(model, params)

In [ ]:
while not generator.is_done():
        generator.compute_logits()
        generator.generate_next_token()

        new_token = generator.get_next_tokens()[0]
        print(tokenizer_stream.decode(new_token), end='', flush=True)
        